## Data Cleaning Summary

### Drop N/A Values from these columns
- delinq_2yrs 
- earliest_cr_line
- inq_last_6mths
- open_acc 
- pub_rec
- total_acc
- revol_util
- purpose
- annual_inc


### Drop Columns
- collections_12_mths_ex_med
    - All values are either 0 or N/A
- pymnt_plan, 
- initial_list_status
    - Huge imbalances, will likely lead to overfitting. Double check.
- mths_since_last_record
    - Large number of N/A values, not immediately clear how to impute. Noted for a later version of the model.
- mths_since_last_delinq
    - Large number of N/A values, not immediately clear how to impute. Noted for a later version of the model.
- zip_code
    - Not informative in model testing, removing for V1.
- addr_state
    - The data in this column is just to sparse for V1.
    
### Drop Outliers/Nonsensical Data
- revol_bal == 1207359, this is an outlier value.
- revol_util > 100

### Simplify/Cleanup Categorical Data
- emp_title (extra)
    - This can be dramatically cleaned up.
- home_ownership, verification_status
    - Simplify

### Distill Commentary (extra)
- Notes
    - Will likely drop for V1.
- purpose
    - Will likely drop and use purpose_cat instead.
- purpose_cat
    - Will choose cutoff count (<100) and map all those values to an 'other' bucket.


## Final Transforms
- Dummies will be created for categorical values.

In [1]:
import datetime
import numpy as np
import pandas as pd
import pickle

df = pd.read_csv('../data/DR_Demo_Lending_Club.csv')
orig_df_length = len(df)

In [2]:
## Create new variable from earliest_cr_line(datetime) = 
## current_date - earliest_cr_line = days since earliest credit line
current_date = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])
df['days_since_earliest_cr_line'] = (current_date - df['earliest_cr_line']).dt.days
df = df.drop('earliest_cr_line', axis=1)

In [3]:
## Convert emp_length to float64 from str.
df.loc[df['emp_length'] == 'na', 'emp_length'] = np.nan
df['emp_length'] = df['emp_length'].astype('float64')

In [4]:
## Drop NA Values from the following columns
drop_na_value_columns = ['delinq_2yrs',
'days_since_earliest_cr_line',
'inq_last_6mths',
'open_acc',
'pub_rec',
'total_acc',
'revol_util',
'purpose',
'annual_inc']

df = df.dropna(axis=0, subset=drop_na_value_columns)
print('Rows dropped: {}'.format(orig_df_length - len(df)))
print('New data length: {}'.format(len(df)))

Rows dropped: 30
New data length: 9970


In [5]:
## Dropping the following for V1 of the model
df = df.drop(['collections_12_mths_ex_med', 
             'pymnt_plan', 
             'initial_list_status',
             'mths_since_last_record',
             'mths_since_last_delinq',
             'zip_code',
             'addr_state'], axis=1)

In [6]:
## Dropping very specific outliers/non-sensical rows
df = df[df['revol_util'] <= 100]

In [7]:
## Creating mapping to consolidate 'VERIFIED - income' and 'VERIFIED - income source' as simply ''VERIFIED - income'
value_map = {
    'VERIFIED - income': 'VERIFIED - income',
    'VERIFIED - income source': 'VERIFIED - income',
    'not verified': 'not verified'
}
df['verification_status'] = [value_map[x] for x in df['verification_status']]

In [8]:
## Mapping lower represented groups to an 'other' bucket
purpose_cat_count = df.groupby(['purpose_cat']).count().sort_values('Id', ascending=False)['Id']
valid_values = list(purpose_cat_count[purpose_cat_count > 100].index)
df['purpose_cat'] = [purpose if purpose in valid_values else 'other' for purpose in df['purpose_cat']]


In [9]:
valid_values

['debt consolidation',
 'credit card',
 'other',
 'home improvement',
 'major purchase',
 'small business',
 'car',
 'wedding',
 'medical',
 'moving',
 'educational',
 'debt consolidation small business']

In [10]:
## Dropping following text columns for V1
df = df.drop(['emp_title', 'Notes','purpose'], axis=1)

In [11]:
## Splitting out categorical data to dummies:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorical_columns = df.select_dtypes(exclude=numerics)

for col in categorical_columns:
    dummies = pd.get_dummies(categorical_columns[col])
    df = df.drop(col, axis=1)
    df = pd.concat([df, dummies], axis=1)
    print(col)

home_ownership
verification_status
purpose_cat
policy_code


In [12]:
## Final df:
df

,Id,is_bad,emp_length,annual_inc,debt_to_income,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,medical,moving,other,small business,wedding,PC1,PC2,PC3,PC4,PC5
0,1,0,10.0,50000.00,10.87,0.0,0.0,15.0,0.0,12087,...,1,0,0,0,0,0,0,0,1,0
1,2,0,1.0,39216.00,9.15,0.0,2.0,4.0,0.0,10114,...,0,0,0,0,0,1,0,0,0,0
2,3,0,4.0,65000.00,11.24,0.0,0.0,4.0,0.0,81,...,0,0,0,0,0,0,0,0,1,0
3,4,0,10.0,57500.00,6.18,1.0,0.0,6.0,0.0,10030,...,0,0,0,0,0,0,1,0,0,0
4,5,0,10.0,50004.00,19.03,0.0,4.0,8.0,0.0,10740,...,0,0,0,0,0,0,0,1,0,0
5,6,0,4.0,47028.00,7.83,2.0,1.0,6.0,0.0,1715,...,0,0,1,0,0,0,0,1,0,0
6,7,0,10.0,126000.00,14.28,0.0,0.0,18.0,0.0,5466,...,0,0,0,0,0,1,0,0,0,0
7,8,0,6.0,42000.00,10.29,0.0,0.0,9.0,0.0,10354,...,0,0,0,0,0,0,0,1,0,0
8,9,0,2.0,50000.00,15.36,0.0,2.0,11.0,0.0,19662,...,0,0,0,0,0,0,0,0,0,1
9,10,0,1.0,40000.00,6.48,0.0,1.0,11.0,0.0,19998,...,0,0,0,0,0,0,0,0,0,1


In [13]:
from library_code import clean_and_transform_data

ImportError: cannot import name 'clean_and_transform_data' from 'library_code' (/Users/jakeweholt/developer/github/DR_TH/analysis/library_code.py)

In [ ]:
df = pd.read_csv('../data/DR_Demo_Lending_Club.csv')
# df = clean_and_transform_data(df)

In [ ]:
df.iloc[0:4]